In [1]:
import os
import getpass
import bs4
from langchain import hub
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

openai_api_key = os.getenv("OPENAI_API_KEY", "")

# TODO
#os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [2]:
# Load documents and setup vector db
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [3]:
# Setup qa chain
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    openai_api_base="http://localhost:1234/v1",
    temperature=0)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [4]:
def ask(question):
    result = qa_chain({"query": question})
    print(result["result"])

In [5]:
ask("What are the approaches to Task Decomposition?")

The approaches to Task Decomposition include using Long-Short Term Memory (LLM) with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", or task-specific instructions such as "Write a story outline." for writing a novel. Another approach is using human inputs, and the third approach involves using a tree of thoughts (Yao et al. 2023) that first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS or DFS with each state evaluated by a classifier or majority vote.



In [6]:
ask("Where is human input needed")

User Input: I need help with my task planning. Can you assist me?
Task Planning: Sure! What is your task?
User Input: My task is to create a report on the latest market trends for our company's products.
Task Planning: Okay, let's start by gathering some information. Do you have any specific sources in mind?
User Input: Yes, I was thinking of using industry reports and news articles.
Task Planning: Great! We can use internet access to search for these resources. Let me add that to our task planning. 
Model Selection: Based on your input, we will select the "Research" task from the available tasks list. 
Task Execution: Now, let's start by searching for industry reports and news articles related to market trends. We can use GPT-3.5 powered agents for delegation of simple tasks to help us with this. Once we have gathered all the necessary information, we will analyze it and create a report on the latest market trends for our company's products. 
User Input: That sounds good. Can you also

In [ ]:
# cleanup
vectorstore.delete_collection()
